In [22]:
"""This work is inspired from Nielsen's algorithm
https://github.com/niconielsen32
"""


"This work is inspired from Nielsen's algorithm\nhttps://github.com/niconielsen32\n"

In [19]:
#imports
import numpy as np
import cv2
import cv2.aruco as aruco
import pathlib
import imutils
import numpy as np
import cv2 as cv, PIL
from cv2 import aruco
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
%matplotlib nbagg
import imutils

In [8]:
def save_coefficients(mtx, dist, path):
    '''Save the camera matrix and the distortion coefficients to given path/file.'''
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
    cv_file.write('K', mtx)
    cv_file.write('D', dist)
    # note you *release* you don't close() a FileStorage object
    cv_file.release()
def load_coefficients(path):
    '''Loads camera matrix and distortion coefficients.'''
    # FILE_STORAGE_READ
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)

    # note we also have to specify the type to retrieve other wise we only get a
    # FileNode object back instead of a matrix
    camera_matrix = cv_file.getNode('K').mat()
    dist_matrix = cv_file.getNode('D').mat()

    cv_file.release()
    return [camera_matrix, dist_matrix]

In [9]:
marker_length = 2.3
square_length = 2.8

In [10]:
#Example of how to create Charuco boards
aruco_dict = aruco.Dictionary_get(aruco.DICT_7X7_50)
board = aruco.CharucoBoard_create(5, 7, square_length, marker_length, aruco_dict)
imboard = board.draw((2000, 2000))
imboard = imutils.resize(imboard, width=600)
cv2.imshow("board",imboard)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite("C:/Users/Ian/Documents/Julien/IMT/Minijob/Anti_counterfeiting_labels/aruco/ressources/posey.jpg",imboard)

True

In [13]:
def calibrate_charuco(dirpath, image_format, marker_length, square_length):
    '''Apply camera calibration using aruco.
    The dimensions are in cm.
    '''
    aruco_dict = aruco.Dictionary_get(aruco.DICT_7X7_50)
    board = aruco.CharucoBoard_create(5, 7, square_length, marker_length, aruco_dict)
    arucoParams = aruco.DetectorParameters_create()

    counter, corners_list, id_list = [], [], []
    img_dir = pathlib.Path(dirpath)
    first = 0
    
    
    # Find the ArUco markers inside each image
    for img in img_dir.glob(f'*{image_format}'):
        print(f'using image {img}')
        image = cv2.imread(str(img))
        image = imutils.resize(image, width=600)
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        corners, ids, rejected = aruco.detectMarkers(
            img_gray, 
            aruco_dict, 
            parameters=arucoParams
        )
        img_detected = aruco.drawDetectedMarkers(image, corners, ids, (0,255,0))
        cv2.imshow("board",image)
        cv2.waitKey()
        cv2.destroyAllWindows()
        resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=img_gray,
            board=board
        )
    
        # If a Charuco board was found, let's collect image/corner points
        # Requiring at least 20 squares
        if resp > 20:
            # Add these corners and ids to our calibration arrays
            corners_list.append(charuco_corners)
            id_list.append(charuco_ids)

    # Actual calibration
    ret, mtx, dist, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=corners_list, 
        charucoIds=id_list, 
        board=board, 
        imageSize=img_gray.shape, 
        cameraMatrix=None, 
        distCoeffs=None)
    
    return [ret, mtx, dist, rvecs, tvecs]

In [14]:
# Parameters
IMAGES_DIR = 'C:/Users/Ian/Documents/Julien/IMT/Minijob/Anti_counterfeiting_labels/aruco/ressources/charuco'
IMAGES_FORMAT = 'jpg'
# Dimensions in cm
MARKER_LENGTH = 2.3
SQUARE_LENGTH = 2.8

# Calibrate 
ret, mtx, dist, rvecs, tvecs = calibrate_charuco(
    IMAGES_DIR, 
    IMAGES_FORMAT,
    MARKER_LENGTH,
    SQUARE_LENGTH
)


UnboundLocalError: local variable 'img_gray' referenced before assignment

In [15]:
save_coefficients(mtx, dist, "C:/Users/Ian/Documents/Julien/IMT/Minijob/Anti_counterfeiting_labels/aruco/ressources/charuco/calibration_charuco.yml")

# Load coefficients
mtx, dist = load_coefficients('C:/Users/Ian/Documents/Julien/IMT/Minijob/Anti_counterfeiting_labels/aruco/ressources/charuco/calibration_charuco.yml')

NameError: name 'mtx' is not defined

In [ ]:
print(mtx)
print(dist)